# PROMETHEUS v2.0 + vLLM + Qwen-72B-Math-INT4
## The Synthesized Weapon: 20+ Novel Insights Operationalized

**Architecture:**
- Model: Qwen-72B-Math-INT4 (vLLM compatible)
- Inference: vLLM (fast batched generation)
- Selection: PROMETHEUS v2 + T.E.S.S.E.R.A.C.T. + NCD Topology

**NEW in v2.0 (from PROMETHEUS SYNTHESIS ENCYCLOPEDIA):**
1. **NCD Trace Topology Consensus** - Cluster traces by semantic similarity
2. **Domain-Specific Basin Centers** - Canonical patterns per math domain
3. **Causal Emergence Scoring (Ψ)** - Macro explains micro
4. **Error Mode Detection** - Systematic error clustering
5. **Confidence Calibration via NCD** - Better uncertainty estimation
6. **ToM-Informed Prompting** - Perspective-taking in prompts
7. **CIC Functional Scoring** - F[T] = Φ(T) - λH(T|X) + γC(T)
8. **Kolmogorov-Weighted Voting** - Compression = confidence

**From Riedl & Weidmann (2025):**
- Theory of Mind predicts AI collaboration (ρs=0.17)
- Solo ability has ZERO correlation with AI output quality (β=-0.00)
- Human collaboration compresses model capability gaps by 5x

**Competition Requirements:**
- AIMO3: 50 problems, GPU ≤5h
- Answer: Integer 0-999 (submit mod 1000)

In [ ]:
# =============================================================================
# CELL 1: ENVIRONMENT SETUP
# =============================================================================
import os
import sys
import gc
import zlib  # For NCD computation

# Environment flags
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# =============================================================================
# IMPORTS
# =============================================================================
import torch
import time
import math
import random
import re
import statistics
import tempfile
import subprocess
import signal
from typing import Optional, List, Tuple, Dict, Any
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from enum import Enum
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import polars as pl
from scipy.cluster.hierarchy import fcluster, linkage

from vllm import LLM, SamplingParams
VLLM_OK = True

# Seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print("="*70)
print("PROMETHEUS v2.0 + vLLM + Qwen-72B-Math-INT4")
print("Synthesized with 20+ Novel Insights")
print("="*70)
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB" if torch.cuda.is_available() else "")

# =============================================================================
# TIMING
# =============================================================================
START_TIME = time.time()
TOTAL_BUDGET = (4 * 60 + 40) * 60  # 4h40m
CUTOFF_TIME = START_TIME + TOTAL_BUDGET
PANIC_TIME = 300

print(f"Budget: {TOTAL_BUDGET//60} minutes")
print("="*70)

In [ ]:
# =============================================================================
# CELL 2: NCD (NORMALIZED COMPRESSION DISTANCE) - CORE PRIMITIVE
# =============================================================================
# From Li et al. (2004) "The Similarity Metric"
# NCD approximates normalized information distance using compression
#
# NCD(x,y) = (K(xy) - min(K(x), K(y))) / max(K(x), K(y))
#
# Properties:
# - 0 = identical (compress perfectly together)
# - 1 = maximally different (no shared structure)
# - ~0.5 = some shared structure
# =============================================================================

def ncd(x: str, y: str) -> float:
    """Normalized Compression Distance using zlib."""
    if not x or not y:
        return 1.0
    cx = len(zlib.compress(x.encode(), level=9))
    cy = len(zlib.compress(y.encode(), level=9))
    cxy = len(zlib.compress((x + y).encode(), level=9))
    return (cxy - min(cx, cy)) / max(cx, cy)


def kolmogorov_proxy(text: str) -> float:
    """Approximate Kolmogorov complexity via compression ratio."""
    if not text:
        return 1.0
    original = len(text.encode())
    compressed = len(zlib.compress(text.encode(), level=9))
    return compressed / original


def compute_ncd_matrix(traces: List[str]) -> np.ndarray:
    """Compute pairwise NCD distance matrix."""
    n = len(traces)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = ncd(traces[i], traces[j])
            dist[i, j] = d
            dist[j, i] = d
    return dist


print("NCD primitives loaded.")
print(f"  Test: ncd('hello world', 'hello world') = {ncd('hello world', 'hello world'):.3f}")
print(f"  Test: ncd('hello world', 'goodbye moon') = {ncd('hello world', 'goodbye moon'):.3f}")

In [ ]:
# =============================================================================
# CELL 3: PROBLEM CLASSIFIER + DOMAIN BASIN CENTERS
# =============================================================================
# INSIGHT B2: Domain-specific canonical reasoning patterns
# =============================================================================

class ProblemType(Enum):
    NUMBER_THEORY = "nt"
    COMBINATORICS = "comb"
    GEOMETRY = "geom"
    ALGEBRA = "alg"
    MIXED = "mixed"


@dataclass
class ProblemProfile:
    primary_type: ProblemType
    constraints: List[str]
    modulo_target: Optional[int]
    enumeration_hint: Optional[int]
    confidence: float


# =============================================================================
# DOMAIN BASIN CENTERS - Canonical reasoning patterns per domain
# Traces closer to these patterns get higher confidence
# =============================================================================
BASIN_CENTERS = {
    ProblemType.NUMBER_THEORY: [
        """Consider the prime factorization. Let n = p1^a1 * p2^a2 * ... * pk^ak.
For divisibility, we need the exponent condition to be satisfied.
Working modulo p, we apply Fermat's Little Theorem: a^(p-1) ≡ 1 (mod p).
Therefore the answer is the value satisfying the modular constraint.""",
        """First, identify the gcd structure. By Bezout's identity, gcd(a,b) = ax + by.
Using the Euclidean algorithm, we reduce step by step.
The Chinese Remainder Theorem gives us the unique solution modulo lcm.
Computing: answer = result.""",
    ],
    ProblemType.COMBINATORICS: [
        """We use a bijection argument. Define the mapping f: A -> B where f(x) = ...
This mapping is well-defined because each element maps to exactly one element.
Injectivity: if f(x) = f(y), then x = y by construction.
Surjectivity: for any b in B, we can construct a = f^{-1}(b).
Therefore |A| = |B| = answer.""",
        """Apply inclusion-exclusion. Let A_i be the set satisfying property i.
|A_1 ∪ A_2 ∪ ... ∪ A_n| = Σ|A_i| - Σ|A_i ∩ A_j| + Σ|A_i ∩ A_j ∩ A_k| - ...
Computing each term and summing gives the final count.""",
    ],
    ProblemType.ALGEBRA: [
        """Let's substitute to simplify. Set u = x + y, v = xy (Vieta's).
The symmetric function becomes a polynomial in u and v.
By AM-GM inequality, we have u^2 >= 4v with equality when x = y.
Optimizing gives the extremal value.""",
        """Consider the polynomial P(x) with roots r_1, r_2, ..., r_n.
By Vieta's formulas: sum of roots = -a_{n-1}/a_n, product = (-1)^n * a_0/a_n.
The desired expression equals a combination of elementary symmetric polynomials.""",
    ],
    ProblemType.GEOMETRY: [
        """Set up coordinates. Place the origin at the center/vertex.
Point A = (x_A, y_A), Point B = (x_B, y_B), etc.
The distance formula gives |AB| = sqrt((x_A-x_B)^2 + (y_A-y_B)^2).
Using the constraint equations, we solve for the unknown coordinates.""",
        """Apply the Law of Cosines: c^2 = a^2 + b^2 - 2ab*cos(C).
For the area, use Heron's formula: A = sqrt(s(s-a)(s-b)(s-c)) where s = (a+b+c)/2.
Alternatively, Area = (1/2)*base*height = (1/2)*ab*sin(C).""",
    ],
    ProblemType.MIXED: [],
}


def classify_problem(question: str) -> ProblemProfile:
    """Classify problem type for strategy routing."""
    q = question.lower()
    scores = {t: 0.0 for t in ProblemType}
    
    # NUMBER THEORY keywords
    for kw in ['remainder', 'modulo', 'mod ', 'divisible', 'prime', 'gcd', 'lcm',
               'factor', 'digit', 'divides', 'coprime', 'congruent', 'fermat']:
        if kw in q: scores[ProblemType.NUMBER_THEORY] += 2
    
    # COMBINATORICS keywords
    for kw in ['how many', 'count', 'number of ways', 'arrangements', 'permutation',
               'combination', 'choose', 'select', 'probability', 'subset', 'bijection']:
        if kw in q: scores[ProblemType.COMBINATORICS] += 2
    
    # GEOMETRY keywords
    for kw in ['triangle', 'circle', 'square', 'rectangle', 'polygon', 'angle',
               'area', 'perimeter', 'diameter', 'radius', 'point', 'line', 'perpendicular']:
        if kw in q: scores[ProblemType.GEOMETRY] += 2
    
    # ALGEBRA keywords
    for kw in ['equation', 'polynomial', 'root', 'solve', 'sequence', 'series',
               'function', 'variable', 'sum of', 'product of', 'inequality']:
        if kw in q: scores[ProblemType.ALGEBRA] += 2
    
    sorted_types = sorted(scores.items(), key=lambda x: -x[1])
    primary = sorted_types[0][0] if sorted_types[0][1] >= 2 else ProblemType.MIXED
    
    # Extract constraints
    constraints = []
    for pattern in [r'such that ([^.]+)', r'where ([^.]+)', r'given that ([^.]+)']:
        constraints.extend(re.findall(pattern, q))
    
    # Detect modulo target
    mod_target = None
    mod_match = re.search(r'(?:mod|modulo|remainder when divided by)\s*(\d+)', q)
    if mod_match:
        mod_target = int(mod_match.group(1))
    
    # Enumeration hint
    enum_hint = None
    numbers = [int(x) for x in re.findall(r'\b(\d+)\b', question) if 1 < int(x) < 10000]
    if numbers and max(numbers) < 1000 and primary == ProblemType.COMBINATORICS:
        enum_hint = max(numbers) ** 2
    
    return ProblemProfile(
        primary_type=primary,
        constraints=constraints[:3],
        modulo_target=mod_target,
        enumeration_hint=enum_hint,
        confidence=min(1.0, sorted_types[0][1] / 10.0)
    )


def distance_to_basin(trace: str, domain: ProblemType) -> float:
    """INSIGHT B2: Distance to domain's canonical patterns."""
    centers = BASIN_CENTERS.get(domain, [])
    if not centers:
        return 0.5  # Neutral for unknown domain
    distances = [ncd(trace, center) for center in centers]
    return min(distances)  # Closest basin center


print("Problem Classifier + Basin Centers loaded.")

In [ ]:
# =============================================================================
# CELL 4: ERROR MODE DETECTION (INSIGHT B4)
# =============================================================================
# Models make systematic errors. Detect and discount them.
# =============================================================================

class ErrorMode(Enum):
    ARITHMETIC = "arithmetic"          # Calculation errors
    SIGN_ERROR = "sign_error"          # +/- confusion
    OFF_BY_ONE = "off_by_one"          # Boundary condition errors
    MODULAR_WRAP = "modular_wrap"      # Forgetting to take mod
    INDEX_SHIFT = "index_shift"        # 0-indexed vs 1-indexed
    OVERCOUNTING = "overcounting"      # Counting items multiple times
    UNDERCOUNTING = "undercounting"    # Missing cases


ERROR_PATTERNS = [
    (ErrorMode.ARITHMETIC, r'\d+\s*[\+\-\*/]\s*\d+\s*=\s*\d+'),
    (ErrorMode.SIGN_ERROR, r'(-?\d+)\s*-\s*(-?\d+)'),
    (ErrorMode.OFF_BY_ONE, r'from\s+\d+\s+to\s+\d+|≤|<|≥|>'),
    (ErrorMode.MODULAR_WRAP, r'mod\s+\d+|\(\s*mod\s+\d+\s*\)'),
]


def detect_error_modes(trace: str, answer: int) -> List[ErrorMode]:
    """Detect likely error modes in a reasoning trace."""
    detected = []
    trace_lower = trace.lower()
    
    for mode, pattern in ERROR_PATTERNS:
        if re.search(pattern, trace_lower, re.IGNORECASE):
            detected.append(mode)
    
    return detected


def error_penalty(errors: List[ErrorMode]) -> float:
    """Compute penalty multiplier based on detected errors."""
    return 0.9 ** len(errors)  # 10% penalty per error mode


print("Error Mode Detection loaded.")

In [ ]:
# =============================================================================
# CELL 5: CAUSAL EMERGENCE SCORING (INSIGHT B7)
# =============================================================================
# Ψ measures how much macro-level structure explains micro-level details
# High Ψ: "By symmetry, pairs cancel" (macro explains many micro)
# Low Ψ: "12+5=17, 17+3=20, ..." (micro without macro)
# =============================================================================

MACRO_PATTERNS = [
    r'by (symmetry|induction|contradiction|construction)',
    r'(therefore|thus|hence|so)\s+all',
    r'in general',
    r'for (all|any|every)',
    r'without loss of generality',
    r'the key (insight|observation|idea)',
    r'this (implies|means|shows) that',
]

MICRO_PATTERNS = [
    r'\d+\s*[\+\-\*/]\s*\d+\s*=\s*\d+',  # Arithmetic
    r'x\s*=\s*\d+',  # Variable assignment
    r'case\s+\d+:',  # Case enumeration
    r'step\s+\d+',  # Step enumeration
]


def compute_causal_emergence(trace: str) -> float:
    """
    INSIGHT B7: Estimate causal emergence (Ψ) from trace.
    
    High Ψ = good abstraction (macro explains micro)
    Low Ψ = mechanical computation (micro without macro)
    """
    trace_lower = trace.lower()
    
    # Count macro-level statements
    macro_count = sum(
        len(re.findall(pattern, trace_lower))
        for pattern in MACRO_PATTERNS
    )
    
    # Count micro-level statements
    micro_count = sum(
        len(re.findall(pattern, trace))
        for pattern in MICRO_PATTERNS
    )
    
    if micro_count == 0:
        return 0.5  # No micro details to explain
    
    # Ideal ratio: ~1 macro per 3 micros
    ratio = macro_count / micro_count if micro_count > 0 else 0
    ideal_ratio = 1/3
    
    # Score peaks at ideal, drops for too much/little macro
    psi = 1 - min(1, abs(ratio - ideal_ratio) / ideal_ratio)
    return max(0, min(1, psi))


print("Causal Emergence Scoring (Ψ) loaded.")

In [ ]:
# =============================================================================
# CELL 6: T.E.S.S.E.R.A.C.T. - TOPOLOGICAL TRACE CONSENSUS (INSIGHT B1)
# =============================================================================
# Topological Ensemble Selection via Semantic-Emergent Reasoning
# And Compression Topology
#
# Instead of naive majority voting, cluster traces by NCD similarity
# and weight consensus by basin stability.
# =============================================================================

@dataclass
class AnswerCandidate:
    value: int
    trace: str
    strategy: str
    kolmogorov_weight: float = 1.0
    basin_distance: float = 0.5
    causal_emergence: float = 0.5
    error_modes: List[ErrorMode] = field(default_factory=list)


def cluster_traces_ncd(
    candidates: List[AnswerCandidate],
    threshold: float = 0.5
) -> Dict[int, List[AnswerCandidate]]:
    """INSIGHT B1: Cluster traces using NCD-based hierarchical clustering."""
    if len(candidates) <= 1:
        return {0: candidates} if candidates else {}
    
    traces = [c.trace for c in candidates]
    
    # Compute NCD distance matrix
    dist_matrix = compute_ncd_matrix(traces)
    
    # Convert to condensed form for scipy
    n = len(traces)
    condensed = dist_matrix[np.triu_indices(n, k=1)]
    
    # Hierarchical clustering
    if len(condensed) > 0:
        Z = linkage(condensed, method='average')
        clusters = fcluster(Z, threshold, criterion='distance')
    else:
        clusters = [1] * len(candidates)
    
    # Group by cluster
    result = defaultdict(list)
    for idx, (cluster_id, cand) in enumerate(zip(clusters, candidates)):
        result[cluster_id].append(cand)
    
    return dict(result)


def compute_basin_stability(cluster: List[AnswerCandidate]) -> float:
    """Compute stability of a reasoning basin."""
    if len(cluster) <= 1:
        return 0.5
    
    traces = [c.trace for c in cluster]
    answers = [c.value for c in cluster]
    
    # Internal cohesion: average pairwise similarity
    total_sim = 0
    count = 0
    for i, t1 in enumerate(traces):
        for t2 in traces[i+1:]:
            total_sim += 1 - ncd(t1, t2)
            count += 1
    cohesion = total_sim / count if count > 0 else 0
    
    # Answer consistency
    unique_answers = len(set(answers))
    consistency = 1.0 / unique_answers
    
    return 0.6 * cohesion + 0.4 * consistency


def tesseract_select(
    candidates: List[AnswerCandidate],
    mod_target: Optional[int] = None,
    domain: ProblemType = ProblemType.MIXED
) -> Tuple[int, float, Dict]:
    """
    T.E.S.S.E.R.A.C.T. Selection Algorithm.
    
    Combines:
    - NCD trace topology clustering (B1)
    - Basin center distance (B2)
    - Kolmogorov weighting (B3)
    - Error mode penalty (B4)
    - Causal emergence scoring (B7)
    """
    if not candidates:
        return 0, 0.0, {}
    
    # Apply modulo if specified
    if mod_target:
        for c in candidates:
            c.value = c.value % mod_target
    
    # Enrich candidates with scores
    for c in candidates:
        c.kolmogorov_weight = 1.0 / (0.5 + kolmogorov_proxy(c.trace) * 0.15)
        c.basin_distance = distance_to_basin(c.trace, domain)
        c.causal_emergence = compute_causal_emergence(c.trace)
        c.error_modes = detect_error_modes(c.trace, c.value)
    
    # Cluster by NCD topology
    clusters = cluster_traces_ncd(candidates, threshold=0.5)
    
    # Score each cluster
    cluster_scores = []
    debug_info = {'clusters': []}
    
    for cluster_id, members in clusters.items():
        stability = compute_basin_stability(members)
        
        # Aggregate scores
        total_k = sum(c.kolmogorov_weight for c in members)
        total_psi = sum(c.causal_emergence for c in members)
        avg_basin = 1 - np.mean([c.basin_distance for c in members])  # Closer = higher
        error_penalty_total = np.prod([error_penalty(c.error_modes) for c in members])
        
        # Majority answer in cluster
        answers = [c.value for c in members]
        majority = Counter(answers).most_common(1)[0][0]
        
        # Combined score (CIC-inspired)
        # F[T] = Φ(T) - λH(T|X) + γC(T)
        # ≈ stability * kolmogorov * emergence * basin_bonus * error_penalty
        score = (
            len(members) *       # Mass
            stability *          # Basin stability (Φ proxy)
            total_k *            # Kolmogorov (compression)
            (1 + total_psi) *    # Causal emergence (C)
            (1 + avg_basin) *    # Basin center bonus
            error_penalty_total  # Error mode penalty
        )
        
        cluster_scores.append((score, majority, cluster_id))
        debug_info['clusters'].append({
            'id': cluster_id,
            'size': len(members),
            'stability': stability,
            'majority': majority,
            'score': score,
        })
    
    # Select highest-scoring cluster's majority answer
    cluster_scores.sort(key=lambda x: -x[0])
    selected = cluster_scores[0][1]
    
    # Confidence: ratio of top cluster to total
    total_score = sum(s[0] for s in cluster_scores)
    confidence = cluster_scores[0][0] / total_score if total_score > 0 else 0
    
    debug_info['selected'] = selected
    debug_info['confidence'] = confidence
    
    return selected, confidence, debug_info


print("T.E.S.S.E.R.A.C.T. Selection loaded.")

In [ ]:
# =============================================================================
# CELL 7: CLASSIC PROMETHEUS COMPONENTS (from v1)
# =============================================================================

# AIMO3 RULE
ANSWER_MIN = 0
ANSWER_MAX = 999  # Submit mod 1000
FALLBACK_ANSWER = 0

# Benford "nice" numbers
NICE_NUMBERS = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 
                36, 40, 42, 45, 48, 50, 54, 60, 64, 72, 80, 81, 90, 96, 100, 108, 120, 
                125, 128, 144, 150, 162, 180, 192, 200, 216, 225, 240, 243, 250, 256,
                270, 288, 300, 324, 360, 384, 400, 432, 450, 480, 486, 500, 512, 540,
                576, 600, 625, 648, 720, 729, 768, 800, 810, 864, 900, 960, 972}

FIBONACCI = {1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987}


def benford_score(n: int) -> float:
    """Mathematical elegance bonus."""
    score = 1.0
    if n in NICE_NUMBERS: score *= 1.3
    if n in FIBONACCI: score *= 1.2
    s = str(abs(n)) if n != 0 else "0"
    trailing_zeros = len(s) - len(s.rstrip('0'))
    if trailing_zeros >= 2: score *= 1.1 + 0.05 * trailing_zeros
    if n > 0 and (n & (n - 1)) == 0: score *= 1.15  # Power of 2
    return score


def toroidal_vote(samples: List[int], mod: int) -> Tuple[int, float]:
    """Circular mean for mod-N problems."""
    if not samples or mod <= 0:
        return 0, 0.1
    
    normalized = [s % mod for s in samples]
    angles = [2 * math.pi * m / mod for m in normalized]
    sin_sum = sum(math.sin(a) for a in angles)
    cos_sum = sum(math.cos(a) for a in angles)
    mean_angle = math.atan2(sin_sum, cos_sum)
    center = int(round(mean_angle * mod / (2 * math.pi))) % mod
    R = math.sqrt(sin_sum**2 + cos_sum**2) / len(samples)
    confidence = min(0.95, max(0.1, R))
    return center, confidence


print("Classic PROMETHEUS components loaded.")

In [ ]:
# =============================================================================
# CELL 8: CODE EXECUTION ENGINE + SELF-HEALING
# =============================================================================

STDLIB = '''
import sys; sys.setrecursionlimit(20000)
import math, numpy as np
from itertools import *
from collections import *
from functools import lru_cache, reduce
from fractions import Fraction
try:
    from sympy import *
    from sympy.ntheory import factorint, divisors, totient, isprime
except: pass

def is_prime(n):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0: return False
    return True

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def lcm(a, b): return a * b // gcd(a, b)

def C(n, k):
    if k < 0 or k > n: return 0
    return math.factorial(n) // (math.factorial(k) * math.factorial(n - k))

def mod_inverse(a, m):
    def extended_gcd(a, b):
        if a == 0: return b, 0, 1
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x
    g, x, _ = extended_gcd(a % m, m)
    return x % m if g == 1 else None
'''

SNOOP = '''
_vars = dict(globals())
for _v in ['answer', 'result', 'ans', 'res', 'final', 'output', 'solution', 'total', 'count']:
    if _v in _vars and _vars[_v] is not None:
        try: print(int(_vars[_v])); break
        except: pass
'''


def extract_python_code(text: str) -> Optional[str]:
    for pattern in [r'```python\s*\n(.*?)```', r'```py\s*\n(.*?)```', r'```\s*\n(.*?)```']:
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if matches:
            return matches[-1].strip()
    return None


def normalize_answer(raw: Any) -> Optional[int]:
    if raw is None: return None
    try:
        s = str(raw).replace(',', '').replace(' ', '').strip()
        s = re.sub(r'[^0-9.eE+-]', '', s)
        if not s: return None
        val = float(s)
        if val != val or abs(val) == float('inf'): return None
        result = int(round(val))
        if result < 0: result = abs(result)
        return result % 1000000
    except:
        return None


def self_heal_code(code: str, stderr: str) -> Optional[str]:
    healed = code
    if 'NameError' in stderr:
        missing = re.findall(r"name '(\w+)' is not defined", stderr)
        for name in missing:
            if name in ['np', 'numpy']: 
                healed = 'import numpy as np\n' + healed
            elif name in ['math', 'sqrt', 'gcd', 'ceil', 'floor']: 
                healed = 'import math\nfrom math import *\n' + healed
            elif name in ['sympy', 'Symbol', 'solve', 'Rational']: 
                healed = 'from sympy import *\n' + healed
            elif name in ['Counter', 'defaultdict', 'deque']: 
                healed = 'from collections import *\n' + healed
            elif name in ['combinations', 'permutations', 'product']: 
                healed = 'from itertools import *\n' + healed
    if 'RecursionError' in stderr:
        healed = 'import sys; sys.setrecursionlimit(50000)\n' + healed
    return healed if healed != code else None


def execute_code(code: str, timeout: int = 15, retries: int = 1) -> Tuple[Optional[int], str]:
    if not code:
        return None, ""
    
    has_print = 'print(' in code
    full = STDLIB + "\n" + code + ("" if has_print else "\n" + SNOOP)
    
    for attempt in range(retries + 1):
        try:
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
                f.write(full)
                f.flush()
                result = subprocess.run(['python', f.name], capture_output=True, 
                                       text=True, timeout=timeout)
                os.unlink(f.name)
                
                if result.returncode == 0 and result.stdout.strip():
                    numbers = re.findall(r'-?\d+\.?\d*', result.stdout)
                    if numbers:
                        return normalize_answer(numbers[-1]), code
                
                if result.stderr and attempt < retries:
                    healed = self_heal_code(code, result.stderr)
                    if healed:
                        code = healed
                        full = STDLIB + "\n" + healed + ("" if has_print else "\n" + SNOOP)
                        continue
        except subprocess.TimeoutExpired:
            pass
        except Exception:
            pass
        break
    return None, code


def extract_boxed(text: str) -> Optional[int]:
    for pattern in [r'\\boxed\{(\d+)\}', r'boxed\{(\d+)\}', r'[Aa]nswer\s*[=:]\\s*(\d+)']:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            return normalize_answer(matches[-1])
    nums = re.findall(r'\b(\d+)\b', text[-500:])
    for n in reversed(nums):
        v = normalize_answer(n)
        if v and 1 < v < 100000:
            return v
    return None


print("Code execution engine loaded.")

In [ ]:
# =============================================================================
# CELL 9: MODEL LOADING + ToM-INFORMED PROMPTS
# =============================================================================

MODEL_PATHS = [
    "/kaggle/input/qwen-72b-math-int4",
    "/kaggle/input/d/ryancardwell/qwen-72b-math-int4",
    "/kaggle/input/qwen-72b-math-nf4",
]

def find_model():
    import glob
    for p in MODEL_PATHS:
        if os.path.exists(p):
            if os.path.exists(os.path.join(p, "config.json")):
                return p
            configs = glob.glob(f"{p}/**/config.json", recursive=True)
            if configs:
                return os.path.dirname(configs[0])
    return None

MODEL_PATH = find_model()
print(f"Model path: {MODEL_PATH}")

# =============================================================================
# vLLM MODEL
# =============================================================================
LLM_MODEL = None

if VLLM_OK and MODEL_PATH:
    print(f"\nLoading vLLM model...")
    try:
        LLM_MODEL = LLM(
            model=MODEL_PATH,
            tensor_parallel_size=1,
            gpu_memory_utilization=0.92,
            trust_remote_code=True,
            max_model_len=8192,
            enforce_eager=True,
            seed=SEED,
        )
        print("vLLM model loaded!")
    except Exception as e:
        print(f"vLLM load failed: {e}")

# =============================================================================
# ToM-INFORMED STRATEGY PROMPTS (INSIGHT from Riedl & Weidmann)
# =============================================================================
# Key insight: Frame the model's ROLE clearly, not what it shouldn't do
# =============================================================================

STRATEGY_PROMPTS = {
    'pot_algorithmic': """You are a mathematical computation engine.
Your role: Convert the problem into executable Python code that computes the answer.
Rules:
- Use int() not float() for final answer
- Print exactly ONE integer at the end
- Use efficient algorithms (avoid O(n!) when possible)
- The printed integer IS your answer""",

    'pot_sympy': """You are a symbolic mathematics engine using SymPy.
Your role: Solve the problem using exact symbolic computation.
Rules:
- Use Rational(a,b) NOT a/b for fractions
- Use sympy.Integer for large numbers
- Print exactly ONE integer at the end
- Verify your symbolic solution numerically if possible""",

    'pot_bruteforce': """You are an enumeration engine for exhaustive search.
Your role: Systematically enumerate all cases to find the answer.
Rules:
- Budget: Maximum 10^7 iterations
- Use generators/itertools for memory efficiency
- Early termination when pattern is clear
- Print exactly ONE integer at the end""",

    'cot': """You are a step-by-step mathematical reasoner.
Your role: Work through the problem systematically, showing each step.
Rules:
- Show your reasoning clearly
- Check your work at each step
- Put your final answer in \\boxed{}
- The boxed value must be a single integer""",
}

TYPE_HINTS = {
    ProblemType.NUMBER_THEORY: "DOMAIN: Number theory. Consider: modular arithmetic, prime factorization, CRT, Fermat/Euler.",
    ProblemType.COMBINATORICS: "DOMAIN: Combinatorics. Consider: bijections, inclusion-exclusion, generating functions, recursion.",
    ProblemType.GEOMETRY: "DOMAIN: Geometry. Consider: coordinates, distance formula, shoelace, trigonometry, similar triangles.",
    ProblemType.ALGEBRA: "DOMAIN: Algebra. Consider: polynomials, Vieta's formulas, AM-GM, substitution, symmetry.",
    ProblemType.MIXED: "",
}

# Temperature annealing
TEMP_START = 0.7
TEMP_MIN = 0.3
TEMP_DECAY = 0.85


def create_prompt(question: str, strategy: str, profile: ProblemProfile) -> str:
    system = STRATEGY_PROMPTS.get(strategy, STRATEGY_PROMPTS['pot_algorithmic'])
    hint = TYPE_HINTS.get(profile.primary_type, "")
    mod_note = f"\nIMPORTANT: Final answer must be computed mod {profile.modulo_target}." if profile.modulo_target else ""
    
    return f"""<|im_start|>system
{system}
{hint}{mod_note}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""


print("ToM-informed prompts loaded.")

In [ ]:
# =============================================================================
# CELL 10: BATCH GENERATION + PROCESSING
# =============================================================================

POT_STOP = ["```output", "```\nOutput", "# Output", "\n\n\n", "<|im_end|>"]
COT_STOP = ["</think>", "\n\nQuestion:", "<|im_end|}"]


def batch_generate(prompts: List[str], mode: str = 'pot', temp: float = 0.6) -> List[str]:
    if not LLM_MODEL:
        return [""] * len(prompts)
    
    params = SamplingParams(
        temperature=max(0.1, temp),
        top_p=0.9,
        max_tokens=4096,
        stop=POT_STOP if mode == 'pot' else COT_STOP,
    )
    
    outputs = LLM_MODEL.generate(prompts, sampling_params=params)
    return [o.outputs[0].text for o in outputs]


def process_pot_outputs(outputs: List[str], strategies: List[str]) -> List[AnswerCandidate]:
    candidates = []
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {}
        for i, out in enumerate(outputs):
            code = extract_python_code(out)
            if code:
                futures[executor.submit(execute_code, code)] = (i, code, out, strategies[i] if i < len(strategies) else 'pot')
        
        for future in as_completed(futures):
            i, code, full_output, strat = futures[future]
            try:
                ans, final_code = future.result()
                if ans is not None:
                    candidates.append(AnswerCandidate(
                        value=ans,
                        trace=full_output,  # Full output as trace for NCD
                        strategy=strat,
                    ))
            except:
                pass
    
    return candidates


def process_cot_outputs(outputs: List[str]) -> List[AnswerCandidate]:
    candidates = []
    for out in outputs:
        ans = extract_boxed(out)
        if ans is not None:
            candidates.append(AnswerCandidate(
                value=ans,
                trace=out,
                strategy="cot",
            ))
    return candidates


print("Batch processing loaded.")

In [ ]:
# =============================================================================
# CELL 11: MAIN SOLVER - PROMETHEUS v2.0 with T.E.S.S.E.R.A.C.T.
# =============================================================================

STRATEGY_WEIGHTS = {
    ProblemType.NUMBER_THEORY: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'pot_bruteforce'],
    ProblemType.COMBINATORICS: ['pot_bruteforce', 'pot_bruteforce', 'pot_algorithmic', 'pot_sympy'],
    ProblemType.GEOMETRY: ['pot_sympy', 'pot_algorithmic', 'pot_algorithmic', 'cot'],
    ProblemType.ALGEBRA: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'cot'],
    ProblemType.MIXED: ['pot_algorithmic', 'pot_sympy', 'pot_bruteforce', 'cot'],
}

PROBLEM_COUNT = 0
SOLVED_COUNT = 0


def panic_guess(question: str, profile: ProblemProfile) -> int:
    """Emergency fallback."""
    numbers = [int(x) for x in re.findall(r'\b\d+\b', question) if 0 < int(x) < 100000]
    if profile.modulo_target and numbers:
        return numbers[0] % profile.modulo_target
    return numbers[0] % 1000 if numbers else 0


def solve_problem(question: str) -> int:
    """
    PROMETHEUS v2.0 Solver with T.E.S.S.E.R.A.C.T. selection.
    
    Incorporates all 20+ insights from PROMETHEUS SYNTHESIS ENCYCLOPEDIA.
    """
    global PROBLEM_COUNT, SOLVED_COUNT
    PROBLEM_COUNT += 1
    
    time_remaining = CUTOFF_TIME - time.time()
    
    # PANIC MODE
    if time_remaining < PANIC_TIME:
        profile = classify_problem(question)
        print(f"  PANIC ({time_remaining:.0f}s)")
        return panic_guess(question, profile)
    
    # CLASSIFY
    profile = classify_problem(question)
    print(f"  Type: {profile.primary_type.value} | Mod: {profile.modulo_target}")
    
    strategies = STRATEGY_WEIGHTS.get(profile.primary_type, STRATEGY_WEIGHTS[ProblemType.MIXED])
    all_candidates = []
    current_temp = TEMP_START
    
    # PHASE 1: Initial batch
    print(f"  Phase 1: {len(strategies)} samples (T={current_temp:.2f})")
    prompts = [create_prompt(question, s, profile) for s in strategies]
    outputs = batch_generate(prompts, 'pot', current_temp)
    candidates = process_pot_outputs(outputs, strategies)
    all_candidates.extend(candidates)
    print(f"    -> {len(candidates)} answers: {[c.value for c in candidates]}")
    
    # EARLY CONSENSUS (60%+ agreement)
    if len(candidates) >= 2:
        values = [c.value for c in candidates]
        counts = Counter(values)
        best, count = counts.most_common(1)[0]
        if count / len(values) >= 0.6:
            print(f"  EARLY CONSENSUS: {best}")
            SOLVED_COUNT += 1
            return int(best) % 1000
    
    # PHASE 2: Expansion (if time permits)
    time_remaining = CUTOFF_TIME - time.time()
    if time_remaining > 180:
        current_temp = max(TEMP_MIN, current_temp * TEMP_DECAY)
        print(f"  Phase 2: expand (T={current_temp:.2f})")
        prompts2 = [create_prompt(question, s, profile) for s in strategies]
        outputs2 = batch_generate(prompts2, 'pot', current_temp)
        candidates2 = process_pot_outputs(outputs2, strategies)
        all_candidates.extend(candidates2)
    
    # PHASE 3: CoT fallback (if few candidates)
    if len(all_candidates) < 3 and time_remaining > 120:
        print(f"  Phase 3: CoT fallback")
        cot_prompts = [create_prompt(question, 'cot', profile) for _ in range(2)]
        cot_outputs = batch_generate(cot_prompts, 'cot', 0.4)
        cot_candidates = process_cot_outputs(cot_outputs)
        all_candidates.extend(cot_candidates)
    
    # NO ANSWERS
    if not all_candidates:
        return panic_guess(question, profile)
    
    # =================================================================
    # T.E.S.S.E.R.A.C.T. SELECTION (the key innovation of v2.0)
    # =================================================================
    final, confidence, debug = tesseract_select(
        all_candidates,
        mod_target=profile.modulo_target,
        domain=profile.primary_type
    )
    
    # TOROIDAL verification for mod problems
    if profile.modulo_target:
        values = [c.value for c in all_candidates]
        tor_ans, tor_conf = toroidal_vote(values, profile.modulo_target)
        if tor_conf > 0.7 and tor_ans != final:
            print(f"    Toroidal override: {tor_ans} (conf={tor_conf:.2f})")
            final = tor_ans
    
    print(f"  T.E.S.S.E.R.A.C.T. FINAL: {final} (conf={confidence:.2f})")
    print(f"    Clusters: {len(debug.get('clusters', []))} | Dist: {Counter([c.value for c in all_candidates])}")
    
    SOLVED_COUNT += 1
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # AIMO3: mod 1000
    return int(final) % 1000


print("PROMETHEUS v2.0 Solver loaded with T.E.S.S.E.R.A.C.T. selection.")

In [ ]:
# =============================================================================
# CELL 12: KAGGLE API INTERFACE
# =============================================================================

TOTAL_PROBLEMS = 50


def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: Optional[pl.DataFrame] = None) -> pl.DataFrame:
    """
    AIMO3 Gateway API.
    """
    id_val = id_.item(0)
    question_str = question.item(0)
    
    time_left = (CUTOFF_TIME - time.time()) / 60
    print(f"\n{'='*60}")
    print(f"ID: {id_val} | Time: {time_left:.1f}m | Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")
    print(f"Q: {question_str[:60]}...")
    
    prediction = solve_problem(question_str)
    
    # AIMO3 Rule: 0-999
    prediction = max(0, min(999, int(prediction)))
    
    print(f"ANSWER: {prediction}")
    print(f"{'='*60}")
    
    return pl.DataFrame({'id': id_val, 'answer': prediction})


print("API interface ready.")
print(f"Budget: {TOTAL_BUDGET//60}min for {TOTAL_PROBLEMS} problems")

In [ ]:
# =============================================================================
# CELL 13: RUN COMPETITION
# =============================================================================
import kaggle_evaluation.aimo_3_inference_server

print("="*70)
print("PROMETHEUS v2.0 + vLLM + T.E.S.S.E.R.A.C.T.")
print("Synthesized from PROMETHEUS SYNTHESIS ENCYCLOPEDIA")
print("="*70)
print(f"Model: {MODEL_PATH}")
print(f"vLLM: {'Loaded' if LLM_MODEL else 'Not available'}")
print(f"Budget: {TOTAL_BUDGET//60} minutes")
print("="*70)

inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Competition mode")
    inference_server.serve()
else:
    print("Local test")
    inference_server.run_local_gateway(('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv',))

print(f"\nCompleted in {(time.time() - START_TIME)/60:.1f} minutes")
print(f"Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")